Instalando dependências

In [1]:
!pip install bibtexparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 51 kB 691 kB/s 
  Created wheel for bibtexparser: filename=bibtexparser-1.4.0-py3-none-any.whl size=42447 sha256=b489822c3b0a3f07f7274f3997edc7f8db46a1e8c30f112c9a6116cd9e76baa2
  Stored in directory: /root/.cache/pip/wheels/0c/b1/30/eea6f0a0ecf1af0d19348e0f6928462817e9176e6b71059dfb
Successfully built bibtexparser


Importando as libs necessárias

In [2]:
import bibtexparser 
import pandas as pd
from google.colab import files
import io
import yaml

#Lendo arquivos de entrada

Carregando arquivos .bib para o notebook

In [3]:
uploaded = files.upload()

Saving acm.bib to acm.bib
Saving acm_2.bib to acm_2.bib
Saving arquivo_config.yaml to arquivo_config.yaml
Saving IEEE_CITATIONS_01.bib to IEEE_CITATIONS_01.bib
Saving ScienceDirect_citations_1665627129854.bib to ScienceDirect_citations_1665627129854.bib


#Criando função que irá ser utilizada durante o código

In [4]:
def junta_arquivos(lista_dfs):
  lista_dfs_temp = None
  for arquivo in lista_dfs:
    if lista_dfs_temp is None:
      lista_dfs_temp = arquivo
    else:
      lista_dfs_temp = lista_dfs_temp.append(arquivo)
  return lista_dfs_temp



# Transformando bibtex em dataframe

In [5]:
files_input = ['acm.bib', 'acm_2.bib','IEEE_CITATIONS_01.bib', 'ScienceDirect_citations_1665627129854.bib']
files_transformed = []

In [6]:
for file_name  in files_input:
  with io.BytesIO(uploaded[file_name]) as bibtex_file:
    bib_database = bibtexparser.load(bibtex_file)
    df = pd.DataFrame(bib_database.entries)
    files_transformed.append(df)

# Analisando nome das colunas de cada arquivo, para entender se elas batem ou não

In [7]:
files_transformed[0].columns 

Index(['series', 'location', 'keywords', 'numpages', 'pages', 'booktitle',
       'abstract', 'doi', 'url', 'address', 'publisher', 'isbn', 'year',
       'title', 'author', 'ENTRYTYPE', 'ID', 'articleno', 'month', 'journal',
       'issn', 'number', 'volume', 'issue_date', 'note'],
      dtype='object')

In [8]:
files_transformed[1].columns

Index(['series', 'location', 'keywords', 'numpages', 'pages', 'booktitle',
       'abstract', 'doi', 'url', 'address', 'publisher', 'isbn', 'year',
       'title', 'author', 'ENTRYTYPE', 'ID', 'articleno', 'month', 'journal',
       'issn', 'number', 'volume', 'issue_date'],
      dtype='object')

In [9]:
files_transformed[2].columns

Index(['month', 'issn', 'doi', 'keywords', 'abstract', 'pages', 'number',
       'volume', 'year', 'title', 'booktitle', 'author', 'ENTRYTYPE', 'ID',
       'journal', 'url', 'isbn', 'publisher'],
      dtype='object')

In [10]:
files_transformed[3].columns

Index(['abstract', 'keywords', 'author', 'url', 'doi', 'issn', 'note', 'year',
       'pages', 'volume', 'journal', 'title', 'ENTRYTYPE', 'ID', 'number',
       'booktitle', 'publisher', 'series', 'editor', 'isbn', 'address',
       'edition'],
      dtype='object')

#Preparando os df para conseguir juntá-los no futuro

Nosso objetivo é deixar as colunas com o seguinte cabeçalho: author, title, keywords, abstract, year, type_publication, doi

Tratando e selecionando apenas as colunas necessárias dos dois arquivos da fonte ACM - https://dl.acm.org/

In [11]:
acm1_file = files_transformed[0][["author", "title", "keywords", "abstract","year", "ENTRYTYPE","doi"]]
acm2_file = files_transformed[1][["author", "title", "keywords", "abstract","year", "ENTRYTYPE","doi"]]

acm_file = acm1_file.append(acm2_file)

Criando a coluna source para identificar de que fonte veio o arquivo

In [12]:
acm_file["source"] = "ACM"

Tratando e selecionando apenas as colunas necessárias do arquivo da fonte IEEE - https://ieeexplore.ieee.org/Xplore/home.jsp

In [13]:
ieee_file = files_transformed[2][["author", "title", "keywords", "abstract","year", "ENTRYTYPE","doi"]]

Criando a coluna source para identificar de que fonte veio o arquivo

In [14]:
ieee_file["source"] = "IEEE"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Tratando e selecionando apenas as colunas necessárias do arquivo da fonte Science Direct - https://www.sciencedirect.com/


In [15]:
science_direct_file = files_transformed[3][["author", "title", "keywords", "abstract","year", "ENTRYTYPE","doi"]]

Criando a coluna source para identificar de que fonte veio o arquivo

In [16]:
science_direct_file["source"] = "ScienceDirect"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Visualizando os arquivos após selecionado as colunas necessárias

In [17]:
display(acm_file)

,author,title,keywords,abstract,year,ENTRYTYPE,doi,source
0,"El Alaoui, Imane and Gahi, Youssef and Messous...",Big Data Quality Metrics for Sentiment Analysi...,"Sentiment analysis, Big data quality metrics, ...","In a world increasingly connected, and in whic...",2019,inproceedings,10.1145/3341620.3341629,ACM
1,"Reda, Oumaima and Sassi, Imad and Zellou, Ahme...",Towards a Data Quality Assessment in Big Data,"Data Quality evaluation, Big Data, Quality Mod...","In recent years, as more and more data sources...",2020,inproceedings,10.1145/3419604.3419803,ACM
2,"Baldassarre, Maria Teresa and Caballero, Ismae...",From Big Data to Smart Data: A Data Quality Pe...,"Big Data, Data Quality, Smart Data",Big Data (BD) solutions are designed to better...,2018,inproceedings,10.1145/3281022.3281026,ACM
3,"Emmanuel, Isitor and Stanier, Clare",Defining Big Data,"Big Data characteristics, Data Quality Dimensi...","As Big Data becomes better understood, there i...",2016,inproceedings,10.1145/3010089.3010090,ACM
4,"Cuzzocrea, Alfredo and Sacc\`{a}, Domenico and...",Big Data: A Research Agenda,"big data, OLAP over big data, privacy of big d...","Recently, a great deal of interest for Big Dat...",2013,inproceedings,10.1145/2513591.2527071,ACM
...,...,...,...,...,...,...,...,...
45,"Zhichao, Xu and Jiandong, Zhao and Huan, Huang",Based on Hadoop's Tech Big Data Combination an...,"tech big data, Hadoop, combination, mining",With the advent of the Internet + era in the f...,2018,inproceedings,10.1145/3194206.3194229,ACM
46,"Wu, Min and Hao, Xinxin and Wan, Xuehong and M...",Opportunities and Challenges of Joint Training...,"big data, cooperative education of university-...",University-industry cooperative education is a...,2022,inproceedings,10.1145/3524383.3524431,ACM
47,"Bizer, Christian and Boncz, Peter and Brodie, ...",The Meaningful Use of Big Data: Four Perspecti...,NaN,Twenty-five Semantic Web and Database research...,2012,article,10.1145/2094114.2094129,ACM
48,"Yu, Bangbo and Zhao, Haijun",Research on the Construction of Big Data Tradi...,"regulatory construction, big data trading plat...","As a new type of asset, the value of big data ...",2019,inproceedings,10.1145/3321454.3321474,ACM


In [18]:
display(ieee_file)

,author,title,keywords,abstract,year,ENTRYTYPE,doi,source
0,"Liu, He and Huang, Fupeng and Li, Han and Liu,...",A Big Data Framework for Electric Power Data Q...,Big Data;Data integrity;Power grids;History;Re...,Since a low-quality data may influence the eff...,2017,inproceedings,10.1109/WISA.2017.29,IEEE
1,"Taleb, Ikbal and Serhani, Mohamed Adel",Big Data Pre-Processing: Closing the Data Qual...,Big Data;Optimization;Data models;Quality asse...,"In the Big Data Era, data is the core for any ...",2017,inproceedings,10.1109/BigDataCongress.2017.73,IEEE
2,"Loetpipatwanich, Sakda and Vichitthamaros, Pre...",Sakdas: A Python Package for Data Profiling an...,Data integrity;Pipelines;Data visualization;Bi...,Data Profiling and data quality management bec...,2020,inproceedings,10.1109/IBDAP50342.2020.9245455,IEEE
3,"Taleb, Ikbal and Serhani, Mohamed Adel and Dss...",Big Data Quality Assessment Model for Unstruct...,Big Data;Data integrity;Data mining;Feature ex...,Big Data has gained an enormous momentum the p...,2018,inproceedings,10.1109/INNOVATIONS.2018.8605945,IEEE
4,"HongJu, Xiao and Fei, Wang and FenMei, Wang an...",Some key problems of data management in army d...,Data engineering;Data analysis;Data integratio...,This paper analyzed the challenges of data man...,2017,inproceedings,10.1109/ICBDA.2017.8078796,IEEE
...,...,...,...,...,...,...,...,...
95,"Wang, Jianwu and Tang, Yan and Nguyen, Mai and...",A Scalable Data Science Workflow Approach for ...,Big data;Engines;Bayes methods;Partitioning al...,"In the Big Data era, machine learning has more...",2014,inproceedings,10.1109/BDC.2014.10,IEEE
96,"Ahnn, Jong Hoon",A Practical Approach to Scalable Big Data Comp...,Adaptation models;Speech;Acoustics;Computation...,We observe that the recent advances in big dat...,2014,inproceedings,10.1109/BDC.2014.11,IEEE
97,"Ludwig, Heiko",Managing Big Data Effectively - A Cloud Provid...,Big data;Cloud computing;Organizations;Face;Co...,Summary form only given. Instrumentation of pr...,2014,inproceedings,10.1109/EDOC.2014.21,IEEE
98,"Ding, Junhua and Kang, Xiaojun and Hu, Xin-Hua...",Building a Deep Learning Classifier for Enhanc...,Diffraction;Big Data;Machine learning;Morpholo...,Providing an easily accessible data service wi...,2017,inproceedings,10.1109/SCC.2017.25,IEEE


In [19]:
display(science_direct_file)

,author,title,keywords,abstract,year,ENTRYTYPE,doi,source
0,Jorge Merino and Ismael Caballero and Bibiano ...,A Data Quality in Use model for Big Data,"Data Quality, Big Data, Measurement, Quality-i...","Beyond the hype of Big Data, something within ...",2016,article,https://doi.org/10.1016/j.future.2015.11.024,ScienceDirect
1,Xintao Liu and Min Chen and Christophe Claramu...,Geographic information science in the era of g...,NaN,NaN,2022,article,https://doi.org/10.1016/j.xinn.2022.100279,ScienceDirect
2,Yuri A.W. Shardt and Xu Yang and Kevin Brooks ...,Data Quality Assessment for System Identificat...,"data quality assessment, system identification...",As the amount of data stored from industrial p...,2020,article,https://doi.org/10.1016/j.ifacol.2020.12.103,ScienceDirect
3,Yi-Chia Lee and Ying-Ting Chao and Pei-Ju Lin ...,Quality assurance of integrative big data for ...,"Big data, Electronic health record, Evidence b...",Background\nThe need is growing to create medi...,2022,article,https://doi.org/10.1016/j.jfma.2021.12.024,ScienceDirect
4,Conor John Cremin and Sabyasachi Dash and Xiao...,Big data: Historic advances and emerging trend...,"Big data, Big data in biomedicine, Data analyt...",Big data is transforming biomedical research b...,2022,article,https://doi.org/10.1016/j.crbiot.2022.02.004,ScienceDirect
...,...,...,...,...,...,...,...,...
95,Angelo Corallo and Anna Maria Crespino and Mar...,Model-based Big Data Analytics-as-a-Service fr...,"Big Data Analytics, BDA, MBDAaaS framework, Sm...","Today, in a smart manufacturing environment ba...",2022,article,https://doi.org/10.1016/j.rcim.2022.102331,ScienceDirect
96,Dirk Schmücker and Julian Reif,Measuring tourism with big data? Empirical ins...,"Big Data, Mobile Network Data, Passive GPS Dat...",In this paper we aim to classify digital data ...,2022,article,https://doi.org/10.1016/j.annale.2022.100061,ScienceDirect
97,Nikolay Nikolov and Yared Dejene Dessalk and A...,Conceptualization and scalable execution of bi...,"Big data workflows, Internet of Things, Domain...","Big Data processing, especially with the incre...",2021,article,https://doi.org/10.1016/j.iot.2021.100440,ScienceDirect
98,Yuncheng Shen and Bing Guo and Yan Shen and Xu...,Personal big data pricing method based on diff...,"Personal big data, Data privacy, Privacy prote...",Personal big data can greatly promote social m...,2022,article,https://doi.org/10.1016/j.cose.2021.102529,ScienceDirect


Juntando os arquivos em apenas um df

In [20]:
arquivos_entrada = junta_arquivos([acm_file, ieee_file, science_direct_file])

In [21]:
display(arquivos_entrada)

,author,title,keywords,abstract,year,ENTRYTYPE,doi,source
0,"El Alaoui, Imane and Gahi, Youssef and Messous...",Big Data Quality Metrics for Sentiment Analysi...,"Sentiment analysis, Big data quality metrics, ...","In a world increasingly connected, and in whic...",2019,inproceedings,10.1145/3341620.3341629,ACM
1,"Reda, Oumaima and Sassi, Imad and Zellou, Ahme...",Towards a Data Quality Assessment in Big Data,"Data Quality evaluation, Big Data, Quality Mod...","In recent years, as more and more data sources...",2020,inproceedings,10.1145/3419604.3419803,ACM
2,"Baldassarre, Maria Teresa and Caballero, Ismae...",From Big Data to Smart Data: A Data Quality Pe...,"Big Data, Data Quality, Smart Data",Big Data (BD) solutions are designed to better...,2018,inproceedings,10.1145/3281022.3281026,ACM
3,"Emmanuel, Isitor and Stanier, Clare",Defining Big Data,"Big Data characteristics, Data Quality Dimensi...","As Big Data becomes better understood, there i...",2016,inproceedings,10.1145/3010089.3010090,ACM
4,"Cuzzocrea, Alfredo and Sacc\`{a}, Domenico and...",Big Data: A Research Agenda,"big data, OLAP over big data, privacy of big d...","Recently, a great deal of interest for Big Dat...",2013,inproceedings,10.1145/2513591.2527071,ACM
...,...,...,...,...,...,...,...,...
95,Angelo Corallo and Anna Maria Crespino and Mar...,Model-based Big Data Analytics-as-a-Service fr...,"Big Data Analytics, BDA, MBDAaaS framework, Sm...","Today, in a smart manufacturing environment ba...",2022,article,https://doi.org/10.1016/j.rcim.2022.102331,ScienceDirect
96,Dirk Schmücker and Julian Reif,Measuring tourism with big data? Empirical ins...,"Big Data, Mobile Network Data, Passive GPS Dat...",In this paper we aim to classify digital data ...,2022,article,https://doi.org/10.1016/j.annale.2022.100061,ScienceDirect
97,Nikolay Nikolov and Yared Dejene Dessalk and A...,Conceptualization and scalable execution of bi...,"Big data workflows, Internet of Things, Domain...","Big Data processing, especially with the incre...",2021,article,https://doi.org/10.1016/j.iot.2021.100440,ScienceDirect
98,Yuncheng Shen and Bing Guo and Yan Shen and Xu...,Personal big data pricing method based on diff...,"Personal big data, Data privacy, Privacy prote...",Personal big data can greatly promote social m...,2022,article,https://doi.org/10.1016/j.cose.2021.102529,ScienceDirect


#Renomeando nome de coluna



In [22]:
arquivos_entrada = arquivos_entrada.rename(columns={'ENTRYTYPE': 'type_publication'})

In [23]:
display(arquivos_entrada)

,author,title,keywords,abstract,year,type_publication,doi,source
0,"El Alaoui, Imane and Gahi, Youssef and Messous...",Big Data Quality Metrics for Sentiment Analysi...,"Sentiment analysis, Big data quality metrics, ...","In a world increasingly connected, and in whic...",2019,inproceedings,10.1145/3341620.3341629,ACM
1,"Reda, Oumaima and Sassi, Imad and Zellou, Ahme...",Towards a Data Quality Assessment in Big Data,"Data Quality evaluation, Big Data, Quality Mod...","In recent years, as more and more data sources...",2020,inproceedings,10.1145/3419604.3419803,ACM
2,"Baldassarre, Maria Teresa and Caballero, Ismae...",From Big Data to Smart Data: A Data Quality Pe...,"Big Data, Data Quality, Smart Data",Big Data (BD) solutions are designed to better...,2018,inproceedings,10.1145/3281022.3281026,ACM
3,"Emmanuel, Isitor and Stanier, Clare",Defining Big Data,"Big Data characteristics, Data Quality Dimensi...","As Big Data becomes better understood, there i...",2016,inproceedings,10.1145/3010089.3010090,ACM
4,"Cuzzocrea, Alfredo and Sacc\`{a}, Domenico and...",Big Data: A Research Agenda,"big data, OLAP over big data, privacy of big d...","Recently, a great deal of interest for Big Dat...",2013,inproceedings,10.1145/2513591.2527071,ACM
...,...,...,...,...,...,...,...,...
95,Angelo Corallo and Anna Maria Crespino and Mar...,Model-based Big Data Analytics-as-a-Service fr...,"Big Data Analytics, BDA, MBDAaaS framework, Sm...","Today, in a smart manufacturing environment ba...",2022,article,https://doi.org/10.1016/j.rcim.2022.102331,ScienceDirect
96,Dirk Schmücker and Julian Reif,Measuring tourism with big data? Empirical ins...,"Big Data, Mobile Network Data, Passive GPS Dat...",In this paper we aim to classify digital data ...,2022,article,https://doi.org/10.1016/j.annale.2022.100061,ScienceDirect
97,Nikolay Nikolov and Yared Dejene Dessalk and A...,Conceptualization and scalable execution of bi...,"Big data workflows, Internet of Things, Domain...","Big Data processing, especially with the incre...",2021,article,https://doi.org/10.1016/j.iot.2021.100440,ScienceDirect
98,Yuncheng Shen and Bing Guo and Yan Shen and Xu...,Personal big data pricing method based on diff...,"Personal big data, Data privacy, Privacy prote...",Personal big data can greatly promote social m...,2022,article,https://doi.org/10.1016/j.cose.2021.102529,ScienceDirect


# Tratando dados para deixar mais padrao para o tipo de coluna (ainda vou construir essa parte)

In [24]:
arquivos_entrada

,author,title,keywords,abstract,year,type_publication,doi,source
0,"El Alaoui, Imane and Gahi, Youssef and Messous...",Big Data Quality Metrics for Sentiment Analysi...,"Sentiment analysis, Big data quality metrics, ...","In a world increasingly connected, and in whic...",2019,inproceedings,10.1145/3341620.3341629,ACM
1,"Reda, Oumaima and Sassi, Imad and Zellou, Ahme...",Towards a Data Quality Assessment in Big Data,"Data Quality evaluation, Big Data, Quality Mod...","In recent years, as more and more data sources...",2020,inproceedings,10.1145/3419604.3419803,ACM
2,"Baldassarre, Maria Teresa and Caballero, Ismae...",From Big Data to Smart Data: A Data Quality Pe...,"Big Data, Data Quality, Smart Data",Big Data (BD) solutions are designed to better...,2018,inproceedings,10.1145/3281022.3281026,ACM
3,"Emmanuel, Isitor and Stanier, Clare",Defining Big Data,"Big Data characteristics, Data Quality Dimensi...","As Big Data becomes better understood, there i...",2016,inproceedings,10.1145/3010089.3010090,ACM
4,"Cuzzocrea, Alfredo and Sacc\`{a}, Domenico and...",Big Data: A Research Agenda,"big data, OLAP over big data, privacy of big d...","Recently, a great deal of interest for Big Dat...",2013,inproceedings,10.1145/2513591.2527071,ACM
...,...,...,...,...,...,...,...,...
95,Angelo Corallo and Anna Maria Crespino and Mar...,Model-based Big Data Analytics-as-a-Service fr...,"Big Data Analytics, BDA, MBDAaaS framework, Sm...","Today, in a smart manufacturing environment ba...",2022,article,https://doi.org/10.1016/j.rcim.2022.102331,ScienceDirect
96,Dirk Schmücker and Julian Reif,Measuring tourism with big data? Empirical ins...,"Big Data, Mobile Network Data, Passive GPS Dat...",In this paper we aim to classify digital data ...,2022,article,https://doi.org/10.1016/j.annale.2022.100061,ScienceDirect
97,Nikolay Nikolov and Yared Dejene Dessalk and A...,Conceptualization and scalable execution of bi...,"Big data workflows, Internet of Things, Domain...","Big Data processing, especially with the incre...",2021,article,https://doi.org/10.1016/j.iot.2021.100440,ScienceDirect
98,Yuncheng Shen and Bing Guo and Yan Shen and Xu...,Personal big data pricing method based on diff...,"Personal big data, Data privacy, Privacy prote...",Personal big data can greatly promote social m...,2022,article,https://doi.org/10.1016/j.cose.2021.102529,ScienceDirect


# Lendo arquivo .yaml de configuração

In [ ]:
with open("arquivo_config.yaml", "r") as stream:
  arquivo_config = yaml.safe_load(stream)
    

In [ ]:
arquivo_config

{'output': ['json', 'csv', 'yaml', 'xml']}

In [ ]:
for tipo in arquivo_config['output']:
  print(tipo)

json
csv
yaml
xml


# Gerando arquivo conforme configuração do .yaml

In [ ]:
#csv
if 'csv' in arquivo_config['output']:
  arquivos_entrada.to_csv('resultado.csv')


In [ ]:
#json
if 'json' in arquivo_config['output']:
  arquivos_entrada.to_json('resultado.json', orient="records")


In [ ]:
#yaml
if 'yaml' in arquivo_config['output']:
  yaml_arquivos_entrada = yaml.dump(arquivos_entrada.to_dict("records"))
  escrita = open("resultado.yaml", "w")
  escrita.write(yaml_arquivos_entrada)
  escrita.close()



In [ ]:
#xml

if 'xml' in arquivo_config['output']:
  arquivos_entrada.to_xml('resultado.xml')
  
